In [5]:
import pandas as pd
import numpy as np

# Reading files in Spark

In [118]:

import ibmos2spark

# @hidden_cell
credentials = {
    'endpoint': 'https://s3-api.us-geo.objectstorage.service.networklayer.com',
    'api_key': 'pGtcK4Cfe0bYhkzj57xpd52SoW35u9AminIgvd2fgQxN',
    'service_id': 'iam-ServiceId-8270013a-9afb-4c42-9b56-f62850a9c0f4',
    'iam_service_endpoint': 'https://iam.ng.bluemix.net/oidc/token'}

configuration_name = 'os_9fa254825cf748af8c99d9684609cb33_configs'
cos = ibmos2spark.CloudObjectStorage(sc, credentials, configuration_name, 'bluemix_cos')

from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
# Please read the documentation of PySpark to learn more about the possibilities to load data files.
# PySpark documentation: https://spark.apache.org/docs/2.0.1/api/python/pyspark.sql.html#pyspark.sql.SparkSession
# The SparkSession object is already initialized for you.
# The following variable contains the path to your file on your IBM Cloud Object Storage.
movies_path = cos.url('movielens_movies.dat', 'sparkscalapythonplayground-donotdelete-pr-pmmafeibz2hwiv')
ratings_path = cos.url('movielens_ratings.dat', 'sparkscalapythonplayground-donotdelete-pr-pmmafeibz2hwiv')
users_path = cos.url('movielens_users.dat', 'sparkscalapythonplayground-donotdelete-pr-pmmafeibz2hwiv')


# Read movies file

In [119]:
movies_df = spark.read.options(delimiter=':').csv(movies_path)

In [120]:
movies_df.take(1)

[Row(_c0='1', _c1=None, _c2='Toy Story (1995)', _c3=None, _c4="Animation|Children's|Comedy")]

In [121]:
movies_df = movies_df.drop('_c1', '_c3')

In [122]:
movies_df.take(1)

[Row(_c0='1', _c2='Toy Story (1995)', _c4="Animation|Children's|Comedy")]

In [123]:
movies_df = movies_df.withColumn('MovieId', movies_df._c0).withColumn('Title', movies_df._c2).withColumn('Genre', movies_df._c4)
movies_df.take(1)

[Row(_c0='1', _c2='Toy Story (1995)', _c4="Animation|Children's|Comedy", MovieId='1', Title='Toy Story (1995)', Genre="Animation|Children's|Comedy")]

In [124]:
movies_df = movies_df.drop('_c0','_c2','_c4')
movies_df.take(1)

[Row(MovieId='1', Title='Toy Story (1995)', Genre="Animation|Children's|Comedy")]

In [125]:
ratings_df = spark.read.options(delimiter=':').csv(ratings_path)
ratings_df.take(1)

[Row(_c0='1', _c1=None, _c2='1193', _c3=None, _c4='5', _c5=None, _c6='978300760')]

In [126]:
ratings_df = ratings_df.drop('_c1','_c3','_c5')
ratings_df.take(1)

[Row(_c0='1', _c2='1193', _c4='5', _c6='978300760')]

In [127]:
ratings_df = ratings_df.withColumn('UserID', ratings_df._c0).withColumn('MoviesId', ratings_df._c2).withColumn('Rating', ratings_df._c4).withColumn('Timestamp', ratings_df._c6)
ratings_df.take(1)

[Row(_c0='1', _c2='1193', _c4='5', _c6='978300760', UserID='1', MoviesId='1193', Rating='5', Timestamp='978300760')]

In [128]:
ratings_df = ratings_df.drop('_c0','_c2','_c4','_c6','UserID_')
ratings_df.take(1)

[Row(UserID='1', MoviesId='1193', Rating='5', Timestamp='978300760')]

In [129]:
users_df = spark.read.options(delimiter=':').csv(users_path)
users_df.take(1)

[Row(_c0='1', _c1=None, _c2='F', _c3=None, _c4='1', _c5=None, _c6='10', _c7=None, _c8='48067')]

In [130]:
users_df = users_df.drop('_c1','_c3','_c5','_c7')
users_df.take(1)

[Row(_c0='1', _c2='F', _c4='1', _c6='10', _c8='48067')]

In [131]:
users_df = users_df.withColumn('UserId', users_df._c0).withColumn('Gender', users_df._c2).withColumn('Age', users_df._c4).withColumn('Occupation', users_df._c6).withColumn('ZipCode', users_df._c8)
users_df.take(1)

[Row(_c0='1', _c2='F', _c4='1', _c6='10', _c8='48067', UserId='1', Gender='F', Age='1', Occupation='10', ZipCode='48067')]

In [132]:
users_df = users_df.drop('_c0','_c2','_c4','_c6','_c8')
users_df.take(1)

[Row(UserId='1', Gender='F', Age='1', Occupation='10', ZipCode='48067')]

In [1]:
age_dict = {"1":  "Under 18",'18':  "18-24",'25':  "25-34",'35':  "35-44",'45':  "45-49",'50':  "50-55", '56':  "56+"}
age_dict

{'1': 'Under 18',
 '18': '18-24',
 '25': '25-34',
 '35': '35-44',
 '45': '45-49',
 '50': '50-55',
 '56': '56+'}

In [2]:
occ_dict = {'0':  "other or not specified", '1':  "academic/educator", '2':  "artist",'3':  "clerical/admin",'4':  "college/grad student",\
            '5':  "customer service",'6':  "doctor/health care",'7':  "executive/managerial",'8':  "farmer",'9':  "homemaker",\
            '10':  "K-12 student",'11':  "lawyer",'12':  "programmer",'13':  "retired",'14':  "sales/marketing",'15':  "scientist",\
            '16':  "self-employed",'17':  "technician/engineer",'18':  "tradesman/craftsman",'19':  "unemployed",'20':  "writer"}
occ_dict

{'0': 'other or not specified',
 '1': 'academic/educator',
 '10': 'K-12 student',
 '11': 'lawyer',
 '12': 'programmer',
 '13': 'retired',
 '14': 'sales/marketing',
 '15': 'scientist',
 '16': 'self-employed',
 '17': 'technician/engineer',
 '18': 'tradesman/craftsman',
 '19': 'unemployed',
 '2': 'artist',
 '20': 'writer',
 '3': 'clerical/admin',
 '4': 'college/grad student',
 '5': 'customer service',
 '6': 'doctor/health care',
 '7': 'executive/managerial',
 '8': 'farmer',
 '9': 'homemaker'}

In [41]:
print(movies_df.take(1))
print(ratings_df.take(1))
print(users_df.take(1))


[Row(MovieId='1', Title='Toy Story (1995)', Genre="Animation|Children's|Comedy")]
[Row(MoviesId='1193', Rating='5', Timestamp='978300760', UserID='1')]
[Row(UserId='1', Gender='F', Age='1', Occupation='10', ZipCode='48067')]


# Which user gave the max number of Rating 1

In [45]:
from pyspark.sql import functions as F

In [53]:
rating_count = ratings_df[ratings_df.Rating == '1'].groupby('UserId').agg(F.count('Rating').alias('Count'))
max_count = rating_count.withColumn('tempId', F.lit(0)).groupBy('tempId').agg(F.max('Count').alias('MaxCount')).select('MaxCount').first()[0]
rating_count.select('UserId', 'Count').where(rating_count.Count == max_count).show()

+------+-----+
|UserId|Count|
+------+-----+
|  4227|  401|
+------+-----+



# Which movie got the max number of 5 rating

In [165]:
#[Row(MovieId='1', Title='Toy Story (1995)', Genre="Animation|Children's|Comedy")]
#[Row(MoviesId='1193', Rating='5', Timestamp='978300760', UserID='1')]

mvi_max5rating = ratings_df[ratings_df.Rating == '5'].groupBy(ratings_df.MoviesId).agg( F.count(ratings_df.MoviesId).alias('Max5Ratings')). \
    sort('Max5Ratings', ascending=False).take(1)
    #take(1).join(movies_df, 'MoviesId' == movies_df.MovieId)
movies_df.select('Title').where(movies_df.MovieId == mvi_max5rating[0][0]).show()

+--------------------+
|               Title|
+--------------------+
|American Beauty (...|
+--------------------+



# People of which occupation gave the maximum number of 3/4/5 ratings to movie American Beauty

In [133]:
american_beauty = movies_df.select('Title', 'MovieId').where(movies_df.Title.like('American Beauty%')).first()
american_beauty_name, american_beauty_id = american_beauty[0], american_beauty[1]

In [171]:
#[Row(MoviesId='1193', Rating='5', Timestamp='978300760', UserID='1')]
#[Row(UserId='1', Gender='F', Age='1', Occupation='10', ZipCode='48067')]

ab_max_ratings = ratings_df[((ratings_df.Rating == 4) | (ratings_df.Rating == 5)) & \
                            (ratings_df.MoviesId == american_beauty_id)][['UserID', 'Rating']]
from pyspark.sql import functions as F
ab_ratings_users = ab_max_ratings.join(users_df, ab_max_ratings.UserID == users_df.UserId)
occupations_count = ab_ratings_users.groupby('Occupation').agg(F.count('Occupation').alias('num_occupations'))
occupation_id = occupations_count.sort('num_occupations', ascending=False).take(1)[0][0]
occ_dict[occupation_id]

'college/grad student'

# Reading files in Pandas

In [ ]:
!wget https://raw.githubusercontent.com/wesm/pydata-book/2nd-edition/datasets/movielens/movies.dat
!wget https://raw.githubusercontent.com/wesm/pydata-book/2nd-edition/datasets/movielens/users.dat
!wget https://raw.githubusercontent.com/wesm/pydata-book/2nd-edition/datasets/movielens/ratings.dat

In [6]:
movies_pdf = pd.read_csv('movies.dat', delimiter='::', header=None)
ratings_pdf = pd.read_csv('ratings.dat', delimiter='::', header=None)
users_pdf = pd.read_csv('users.dat', delimiter='::', header=None)
print(movies_pdf.head(1))
print(ratings_pdf.head(1))
print(users_pdf.head(1))

/usr/local/src/conda3_runtime/home/envs/DSX-Python35-Spark/lib/python3.5/site-packages/ipykernel/__main__.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  if __name__ == '__main__':
/usr/local/src/conda3_runtime/home/envs/DSX-Python35-Spark/lib/python3.5/site-packages/ipykernel/__main__.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  from ipykernel import kernelapp as app


   0                 1                            2
0  1  Toy Story (1995)  Animation|Children's|Comedy
   0     1  2          3
0  1  1193  5  978300760
   0  1  2   3      4
0  1  F  1  10  48067


/usr/local/src/conda3_runtime/home/envs/DSX-Python35-Spark/lib/python3.5/site-packages/ipykernel/__main__.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  app.launch_new_instance()


In [7]:
movies_pdf.columns=['MovieId', 'Title', 'Genre']
movies_pdf.head(1)

,MovieId,Title,Genre
0,1,Toy Story (1995),Animation|Children's|Comedy


In [8]:
ratings_pdf.columns = ['UserID','MovieID','Rating','Timestamp']
ratings_pdf.head(1)

,UserID,MovieID,Rating,Timestamp
0,1,1193,5,978300760


In [9]:
users_pdf.columns = ['UserID','Gender','Age','Occupation','Zip-code']
users_pdf.head(1)

,UserID,Gender,Age,Occupation,Zip-code
0,1,F,1,10,48067


# People of which occupation gave the maximum number of 3/4/5 ratings to movie American Beauty

In [18]:
ab_id_nm = movies_pdf[movies_pdf.Title.str.startswith('American Beauty')][['MovieId', 'Title']]
ab_id_nm

,MovieId,Title
2789,2858,American Beauty (1999)


In [144]:
occ_id = ratings_pdf[(ratings_pdf.Rating == 3) | (ratings_pdf.Rating == 4) | (ratings_pdf.Rating == 5)][(ratings_pdf.MovieID == ab_id_nm.MovieId.iloc[0])]. \
merge(users_pdf,).Occupation.value_counts().index[0]
occ_dict[str(occ_id)]

/usr/local/src/conda3_runtime/home/envs/DSX-Python35-Spark/lib/python3.5/site-packages/ipykernel/__main__.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if __name__ == '__main__':


'college/grad student'